In [ ]:
!pip install SPARQLWrapper
!pip install pandas
!pip install numpy
!pip install rdflib

In [33]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML, TURTLE, N3, RDF, RDFXML, CSV, TSV, JSONLD, DIGEST
import pandas as pd
import time
from random import randint
import numpy as np
import requests
from requests.exceptions import HTTPError
from rdflib import Graph

In [1]:
prefix = """
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX schema: <http://schema.org/>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX dbc: <http://dbpedia.org/resource/Category:>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbt: <http://dbpedia.org/resource/Template:>
    PREFIX dbyago: <http://dbpedia.org/class/yago/>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX gnd: <http://d-nb.info/gnd/>
    PREFIX gold: <http://purl.org/linguistics/gold/>
    PREFIX prov: <http://www.w3.org/ns/prov#>
    PREFIX umbelrc: <http://umbel.org/umbel/rc/>
    PREFIX viaf: <http://viaf.org/viaf/>
    PREFIX ore: <http://www.openarchives.org/ore/terms/>
    PREFIX edm: <http://www.europeana.eu/schemas/edm/>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdata: <http://www.wikidata.org/wiki/Special:EntityData/>
    PREFIX wdno: <http://www.wikidata.org/prop/novalue/>
    PREFIX wdref: <http://www.wikidata.org/reference/>
    PREFIX wds: <http://www.wikidata.org/entity/statement/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX wdtn: <http://www.wikidata.org/prop/direct-normalized/>
    PREFIX wdv: <http://www.wikidata.org/value/>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    PREFIX eventKG-s: <http://eventKG.l3s.uni-hannover.de/schema/>
    PREFIX eventKG-e: <http://eventKG.l3s.uni-hannover.de/resource/>
"""


In [ ]:
def first_query(sparqlurl, credentials):
  sparql = SPARQLWrapper(sparqlurl)
  sparql.setCredentials(credentials[0], credentials[1])
  #sparql.setCredentials("intavia", "acdhresearchspace2021")
  #sparql.setCredentials("intavia", "gE1aG9xK2hS4uQ4j")
  query_content = """
  select *
  where {
    ?s ?p ?o
  }
  LIMIT 10
  """

  query = query_content 
  sparql.setQuery(query)
  #sparql.setReturnFormat(TURTLE)
  sparql.setReturnFormat(JSON)
  #sparql.setReturnFormat(RDFXML)
  results = sparql.query().convert()
  print(results)
  # # Create an empty dataframe, and append data to the dataframe in each iteration
  df = pd.DataFrame(columns=['s', 'p', 'o'])
  for result in results["results"]["bindings"]:
    #print(result)
    s = result["s"]["value"]
    p = result["p"]["value"]
    o = result["o"]["value"]
    dicts = {'s': s, 'p': p, 'o': o}
    df = df.append(dicts, ignore_index=True, sort=False)
  # #print('-----------------')
  # #print(df.iloc[0:3])
  return df

sparqlurl = 'https://mp-playground.acdh-dev.oeaw.ac.at/sparql'
#sparql = SPARQLWrapper("https://triplestore.acdh-dev.oeaw.ac.at/intavia/sparql")
credentials = ["intavia", "acdhresearchspace2021"]
#credentials = ["intavia", "gE1aG9xK2hS4uQ4j"]
df = first_query(sparqlurl, credentials)
print(df)

# Method 2: HTTP request and RDFlib (without SPARQL)

In [73]:
#url = 'https://dbpedia.org/resource/Gustav_Klimt'
#url = 'http://data.europeana.eu/item/2023859/_http___keptar_oszk_hu_025900_025984__'
#url = 'http://eventKG.l3s.uni-hannover.de/resource/entity_3478776'

url = 'http://data.biographynet.nl/rdf/PersonDes-10008408_02'
# InTaVia, Finnish, APIS URIs do not work for content negotiation
#url = 'http://www.intavia.eu/personproxy/bs/4943'
#url = 'http://www.intavia.eu/apis/personproxy/100029'
#url = 'https://apis.acdh.oeaw.ac.at/apis/api/entities/person/102112/'

headers = {
    #'Content-type': 'text/html; charset=UTF-8',
    'Accept': 'text/turtle', 
    'Content-type': 'text/turtle',
    #'Accept': 'application/rdf+xml',
    #'Content-type': 'application/rdf+xml',
}

#r = requests.get(url)
r = requests.get(url, headers=headers)
try: 
    r.raise_for_status()
except requests.exceptions.HTTPError as e: 
    print(e)
print('-------')
print(r)
header = r.headers
print(header)
print('-------')

contentType = r.headers['content-type']
print(contentType)
print('-------')

print('-------')
print(r.content)
print('-------')
print(r.text)

# Create a new file to save as HTML (don't get confused with name "open")
#with open('test.html', 'w') as f:
#    f.write(r.text)

406 Client Error: Not Acceptable for url: https://apis.acdh.oeaw.ac.at/apis/api/entities/person/102112/
-------
<Response [406]>
{'Date': 'Wed, 06 Jul 2022 09:54:42 GMT', 'Server': 'nginx/1.22.0', 'Allow': 'GET, PUT, PATCH, DELETE, HEAD, OPTIONS', 'Content-Length': '57', 'Content-Type': 'application/json', 'Vary': 'Accept,Origin', 'X-Frame-Options': 'SAMEORIGIN', 'Via': '1.1 apis.acdh.oeaw.ac.at', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive'}
-------
application/json
-------
-------
b'{"detail":"Could not satisfy the request Accept header."}'
-------
{"detail":"Could not satisfy the request Accept header."}


In [75]:
#url = 'https://dbpedia.org/resource/Gustav_Klimt'
#url ='http://data.europeana.eu/item/9200333/BibliographicResource_3000058955043'
# No difference between URIs (item, providerProxy etc)
#url ='http://data.europeana.eu/proxy/provider/9200333/BibliographicResource_3000058955043'
# Obsolete URI found in a result in SPARQL 
#url = 'http://data.europeana.eu/item/2048212/zac_posen_aw13_0072'
#url = 'http://eventKG.l3s.uni-hannover.de/resource/entity_3478776'

url = 'http://data.biographynet.nl/rdf/PersonDes-10008408_02'
# InTaVia, Finnish, APIS URIs do not work for content negotiation
#url = 'http://www.intavia.eu/personproxy/bs/4943'
#url = 'http://www.intavia.eu/apis/personproxy/100029'
#url = 'https://apis.acdh.oeaw.ac.at/apis/api/entities/person/102112/'

g = Graph()
g.parse(url)
#g.parse(url, format="turtle")
print(g)
# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("It better be!")
print('--------')
# Print the number of "triples" in the Graph
print(f"Graph g has {len(g)} statements.")
print('--------')
# Print out the entire Graph in the RDF Turtle format
print(g.serialize(format="xml"))
print('--------')

# Save as ttl file
#g.serialize(destination="test.ttl")
# Read from ttl file
#g.parse('test.ttl', format='ttl')
# Print out the entire Graph in the RDF/XML format
#print(g.serialize(format="xml"))

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].
--------
Graph g has 20 statements.
--------
<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:bgn="http://data.biographynet.nl/rdf/"
   xmlns:ore="http://www.openarchives.org/ore/terms/"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:about="http://data.biographynet.nl/rdf/PersonDes-10008408_02">
    <rdf:type rdf:resource="http://data.biographynet.nl/rdf/PersonDes"/>
    <rdf:type rdf:resource="http://www.openarchives.org/ore/terms/Proxy"/>
    <bgn:event rdf:nodeID="N3e867a76c3d34a8595631a8ec11293db"/>
    <bgn:event rdf:nodeID="N0bdadcad318b446bb21a8701297ecaca"/>
    <bgn:idno>10008408</bgn:idno>
    <bgn:idno>raa</bgn:idno>
    <bgn:idno>raa/11127.xml</bgn:idno>
    <bgn:personName rdf:nodeID="N90da495849004e1bb6911237b0a64083"/>
    <ore:proxyFor rdf:resource="http://data.biographynet.nl/rdf/Person-10008408"/>
    <ore:proxyIn rdf:resource="http://data.biographynet.nl/r

In [ ]:
#URI_list_df = print(URI_list_df)
#Saving URI list as CSV file (will be lost after the session)
filename = 'TEST_EventKG_df.csv'
df_csv = URI_list_df.to_csv(filename)
#Saving URI list in GoogleDrive (which should be mounted by clicking icon in file structure view)
#df_csv = URI_list_df.to_csv('/content/drive/MyDrive/Colab Notebooks/InTaVia/' + filename)